<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
#библиотеки проекта
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.utils import shuffle


import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    df = pd.read_csv('Churn.csv')
except:
    df = pd.read_csv('/datasets/Churn.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
df.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [5]:
#проверим наличие явных дубликатов
df.duplicated().sum()

0

В результате короткого обзора и оценки данных предпринимаем следующее решение:
1. Столбцы 'RowNumber', 'CustomerId', 'Surname' удаляем  - их содержание не влияет на исследование и обучение модели.
2. Пропуски в столбце 'Tenure' удаляем. Да, их около 10%, но данные не восстановить, а замена на нули или медианой может повлиять на результат обучения. Признак продолжительности лояльности клиента банку, на мой взгляд важный. Качество важнее количества.
3. И, строго соблюдая традиции, приводим регистр названий колонок к строчному и "змеиному")

In [6]:
#удалим столбцы
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
#проверим
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [7]:
#удаляем пропуски
df = df.dropna(subset=['Tenure'])
#проверим
df['Tenure'].isnull().sum()

0

In [8]:
#приводим названия к строчным
df.columns = df.columns.str.lower()
#приводим к змеиному регистру
df = df.rename(columns={'creditscore':'credit_score',
                                  'numofproducts':'num_of_products',
                                  'hascrcard':'has_cr_card',
                                  'isactivemember':'is_active_member',
                                  'estimatedsalary': 'estimated_salary'
                                 })
#проверяем
df.columns

Index(['credit_score', 'geography', 'gender', 'age', 'tenure', 'balance',
       'num_of_products', 'has_cr_card', 'is_active_member',
       'estimated_salary', 'exited'],
      dtype='object')

In [33]:
#сейчас все ок
df.head()

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [10]:
#посмотрим на распределение классов в целевом признаке
df['exited'].value_counts(normalize=True)

0    0.796062
1    0.203938
Name: exited, dtype: float64

Около 20% клиентов уходят из банка. Приличное количество, но нам надо работать с прогнозированием оттока лояльных клиентов. Долю объектов положительного класса скорее всего надо будет увеличивать.

In [11]:
#оценим данные на мультиколлинеарность
df.corr()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
credit_score,1.000000,-0.004504,-0.000062,0.002804,0.011090,-0.003937,0.030947,0.005182,-0.023950
age,-0.004504,1.000000,-0.013134,0.031481,-0.031877,-0.014129,0.082269,-0.007037,0.283017
tenure,-0.000062,-0.013134,1.000000,-0.007911,0.011979,0.027232,-0.032178,0.010520,-0.016761
balance,0.002804,0.031481,-0.007911,1.000000,-0.301858,-0.019055,-0.003285,0.014351,0.117218
num_of_products,0.011090,-0.031877,0.011979,-0.301858,1.000000,0.005805,0.009084,0.013990,-0.050271
has_cr_card,-0.003937,-0.014129,0.027232,-0.019055,0.005805,1.000000,-0.008810,-0.006136,-0.005411
is_active_member,0.030947,0.082269,-0.032178,-0.003285,0.009084,-0.008810,1.000000,-0.020049,-0.155062
estimated_salary,0.005182,-0.007037,0.010520,0.014351,0.013990,-0.006136,-0.020049,1.000000,0.016029
exited,-0.023950,0.283017,-0.016761,0.117218,-0.050271,-0.005411,-0.155062,0.016029,1.000000


Корреляция с целевым признаком очень слабая, ее почти нет. Зависимость в данных вероятно сложнее.

In [35]:
#смотрим на распределение количественных данных
df.describe()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
count,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000,9091.000000
mean,650.736553,38.949181,4.997690,76522.740015,1.530195,0.704983,0.515565,100181.214924,0.203938
std,96.410471,10.555581,2.894723,62329.528576,0.581003,0.456076,0.499785,57624.755647,0.402946
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,2.000000,0.000000,1.000000,0.000000,0.000000,51227.745000,0.000000
50%,652.000000,37.000000,5.000000,97318.250000,1.000000,1.000000,1.000000,100240.200000,0.000000
75%,717.000000,44.000000,7.000000,127561.890000,2.000000,1.000000,1.000000,149567.210000,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


Выбросов в данных нет. Max и Min значения вполне адекватны.

Типы данных в для исследования нам нужны количественные, все признаки ок, кроме 'gender' и 'geography'.
       

In [13]:
#преобразуем категориальные признаки в численные прямым кодированием
df_ohe = pd.get_dummies(df, columns = ['gender', 'geography'], drop_first=True)
#проверим
df_ohe.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,gender_Male,geography_Germany,geography_Spain
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,0,1


Вывод часть 1. 
Мы провели подготовку данных для исследования. Были удалены колонки, содержание которых не влияет на исследование и тестирование будущей модели. Удалили пропуски в 'tenure'. Названия признаков приведены к нижнему регистру и 'змеиному' стилю. Было установлено, что явных дубликатов и выбросов в данных нет. Корреляция данных с целевым признаком очень слабая, возможно зависимость гораздо сложнее. Также преобразовали категориальные признаки в количественные. Переходим к исследованию задачи.

## Исследование задачи

In [14]:
#объявим переменные для признаков и целевого признака
features = df_ohe.drop(['exited'], axis=1)
target = df_ohe['exited']

#разделим сет на обучающую и валидационную выборки в соотношении 60/40
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target, test_size=0.4,
                                                                              random_state=12345)

#разделим валидационную выборку на валидационную и тестовую в соотношении 20/20
features_test, features_valid, target_test, target_valid = train_test_split(features_valid,
                                                                            target_valid, test_size=0.5,
                                                                            random_state=12345)

#проверим размеры
for i in [features_train, features_valid, target_train, target_valid, features_test, target_test]:
    print(i.shape)

(5454, 11)
(1819, 11)
(5454,)
(1819,)
(1818, 11)
(1818,)


Проведем обучение моделей без учета дисбаланса

In [15]:
#DecisionTreeClassifier
best_f1_dtc = 0
best_depth_dtc = 0
best_auc_roc = 0
for depth in range(1, 15):
    dtc = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    dtc.fit(features_train, target_train)
    predicted_valid_dtc = dtc.predict(features_valid)
    f1_dtc = f1_score(target_valid, predicted_valid_dtc)
    auc_roc_dtc = roc_auc_score(target_valid, dtc.predict_proba(features_valid)[:, 1])
    if f1_dtc > best_f1_dtc:
        best_f1_dtc = f1_dtc
        best_depth_dtc = depth
        best_auc_roc = auc_roc_dtc
        
print('F1 для дерева решений =', best_f1_dtc)
print('Максимальная глубина =', best_depth_dtc)
print('AUC-ROC =', best_auc_roc)

F1 для дерева решений = 0.5559322033898305
Максимальная глубина = 7
AUC-ROC = 0.8305629303585707


In [16]:
#RandomForestClassifier
best_f1_rfc = 0
best_depth_rfc = 0
best_est_rfc = 0
best_roc_auc_score_rfc = 0
for depth in range(1, 10):
    for est in range(1, 100):
        rfc = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        rfc.fit(features_train, target_train)
        predicted_valid_rfc = rfc.predict(features_valid)
        f1_rfc = f1_score(target_valid, predicted_valid_rfc)
        roc_auc_score_rfc = roc_auc_score(target_valid, rfc.predict_proba(features_valid)[:,1])
        if f1_rfc > best_f1_rfc:
            best_f1_rfc = f1_rfc
            best_depth_rfc = depth
            best_est_rfc = est
            best_roc_auc_score_rfc = roc_auc_score_rfc
            
print('F1 для случайного леса =', best_f1_rfc)
print('Максимальная глубина =', best_depth_rfc)
print('Число деревьев =', best_est_rfc)
print('AUC-ROC =', best_roc_auc_score_rfc)

F1 для случайного леса = 0.5595667870036101
Максимальная глубина = 9
Число деревьев = 13
AUC-ROC = 0.8476433622891388


Вывод часть 2. Похожие значения F1-меры наблюдаются в моделях DecisionTreeClassifier, RandomForestClassifier - 0.5559 и 0.5596 соответственно. Результат AUC-ROC - 0.8306 и 0.8476 соответственно. Поскольку разница небольшая бороться с дисбалансом назначены обе модели. 

## Борьба с дисбалансом

Метод Upsample

In [25]:
#увеличим число наблюдений положительного класса:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled,\
                                                   target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

#для DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_upsampled, target_upsampled) 
predicted_valid = model.predict(features_valid)

print('Для дерева решений на увеличенной выборке F1 =', f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))

Для дерева решений на увеличенной выборке F1 = 0.5400192864030857
AUC-ROC = 0.8285300076853208


In [26]:
#для RandomForestClassifier
model = RandomForestClassifier(random_state=12345, n_estimators=13, max_depth=9)
model.fit(features_upsampled, target_upsampled) 
predicted_valid = model.predict(features_valid)

print('Для случайного леса на увеличенной выборке F1 =', f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))

Для случайного леса на увеличенной выборке F1 = 0.5787631271878647
AUC-ROC = 0.8493356855073475


Метод Downsample

In [27]:
#уменьшим число наблюдений отрицательного класса:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)]\
                                     + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)]\
                                   + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(features_downsampled,\
                                                       target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

#для DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=12345, max_depth=7)
model.fit(features_downsampled, target_downsampled) 
predicted_valid = model.predict(features_valid)

print('Для дерева решений на увеличенной выборке F1 =', f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))

Для дерева решений на увеличенной выборке F1 = 0.5490584737363726
AUC-ROC = 0.8279749567215509


In [28]:
#для RandomForestClassifier
model = RandomForestClassifier(random_state=12345, n_estimators=13, max_depth=9)
model.fit(features_downsampled, target_downsampled) 
predicted_valid = model.predict(features_valid)

print('Для случайного леса на увеличенной выборке F1 =', f1_score(target_valid, predicted_valid))
print('AUC-ROC =', roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))

Для случайного леса на увеличенной выборке F1 = 0.5665601703940363
AUC-ROC = 0.8457899578471786


Вывод часть 3. Выбираем модель RandomForestClassifier с показателями F1 - 0.5788 и AUC-ROC - 0.8493. Мы довели метрику F1 до заданного значения методом Upsample. При использовании метода Downsample (уменьшение выборки отрицательных классов) показатели метрик F1 и AUC-ROC меньше.
Переходим к тестированию.

## Тестирование модели

In [21]:
#объединим для обучения данные train и valid:
features_train_valid = pd.concat([features_train] + [features_valid])
target_train_valid = pd.concat([target_train] + [target_valid]) 

In [22]:
#метод upsampling применим к объединенной выборке:
features_upsampled, target_upsampled = upsample(features_train_valid, target_train_valid, 4)

In [32]:
#тестируем RandomForestClassifier:
model_rfc_test = RandomForestClassifier(random_state=12345, n_estimators=13, max_depth=9)
model_rfc_test.fit(features_upsampled, target_upsampled)
predictions_rfc_test = model_rfc_test.predict(features_test)

print('F1 =', f1_score(target_test, predictions_rfc_test))
print('AUC-ROC =', roc_auc_score(target_test, model_rfc_test.predict_proba(features_test)[:,1]))

F1 = 0.6233480176211453
AUC-ROC = 0.8584795041757399


Вывод часть 4.
Проверка на тестовой выборке подтвердила, что метод Upsampling помог справиться с дисбалансом классов, показатели увеличились до F1 = 0.6233, AUC-ROC = 0.8485

Общий вывод. В датасете содержались данные о клиентах банка с определенными признаками и целевым признаком 'exited'.  Мы подготовили данные к исследованию. Удалили признаки не влияющие на обучение и пропуски значений. Привели названия к нижнему регистру и изменили тип данных, где это требовалось, на мой взгляд. Оценили корреляцию признаков и выявили дисбаланс данных в целевом признаке.
При исследовании и обучении моделей с дисбалансом с лучшим показателем лидировала модель RandomForestClassifier с метриками F1 - 0.5596, AUC-ROC 0.8476. После применения метода Upsample (увеличения числа наблюдений положительного класса) показатели метрик выросли до F1 - 0.5788, AUC-ROC - 0.8493. Проверка на тестовой выборке подтвердила, что метод Upsampling помог справиться с дисбалансом классов показатели выросли до F1 = 0.6233, AUC-ROC = 0.8485.
Учитывая, что F1 - среднее гармоническое полноты и точности, то рост F1 свидетельствует об изменении в алгоритме в лучшую сторону. Также рост AUC-ROC (площадь под кривой) подтверждает улучшение модели (стремится к 1). 
Мы построили оптимальную модель с предельно большим значением F1-меры для прогнозирования оттока клиента банка в ближайшее время.